In [51]:
import pandas as pd
import numpy as np
import plotly.express as px
import pandas as pd
import utilities.utility as utility
import plotly.graph_objects as go

In [52]:
df = pd.read_csv('../data/10_monthly_returns_complete.csv')
df_close = pd.read_csv('../data/10_monthly_adjacent_close.csv', index_col=0)
df_overview = pd.read_csv('../data/data_10_overview.csv', index_col=0)

In [53]:
df_overview['return_rate_1y_avg'] = df_overview['return_rate_1y_avg'] - 1
df_overview['return_rate_5y_avg'] = df_overview['return_rate_5y_avg'] - 1
df_overview['return_rate_10y_avg'] = df_overview['return_rate_10y_avg'] - 1
df_overview['return_rate_25y_avg'] = df_overview['return_rate_25y_avg'] - 1

In [54]:
df_overview = df_overview.dropna()

## Equities: Risk-Return

In [55]:
df_risk_return = df_overview.sort_values(by='market_capital_euro', ascending=False)
df_risk_return = df_risk_return[['stock_ticker_symbol', 'return_rate_25y_avg', 'return_rate_5y_avg', 'return_rate_10y_avg', 'return_rate_1y_avg', 'volatility_5y']]
# Create a scatter plot
fig = px.scatter(df_risk_return, 
                 x='volatility_5y', 
                 y='return_rate_5y_avg', 
                 text='stock_ticker_symbol',
                 title='Risk-Return Plot',
                 labels={'volatility_5y': 'Risk (Standard Deviation)',
                         'return_rate_5y_avg': 'Investment Return Average (last 5 years)'},
                 template='plotly')

# Add labels to the points
fig.update_traces(textposition='top center')

# Show the plot
fig.show()

## Equities: Efficient-Frontier

In [ ]:
def plot_efficient_frontier(df):
    target_returns, portfolio_volatilities = utility.efficient_frontier(df)

    # Create the figure
    fig = go.Figure()

    # Add the Efficient Frontier line
    fig.add_trace(go.Scatter(
        x=portfolio_volatilities,
        y=target_returns,
        mode='lines',
        name='Efficient Frontier',
        line=dict(color='green', width=3)
    ))

    # Add the individual assets as points
    fig.add_trace(go.Scatter(
        x=df['volatility_5y'],
        y=df['return_rate_5y_avg'],
        mode='markers',
        name='Assets',
        fillcolor='purple'
    ))

    # Update layout
    fig.update_layout(
        title='Efficient Frontier',
        xaxis_title='Volatility (Risk)',
        yaxis_title='Expected Return',
        legend=dict(x=0, y=1),
        hovermode='closest',
        template='plotly'
    )

    # Add labels to the points
    fig.update_traces(textposition='top center')
    
    fig.show()

# Plot the Efficient Frontier
plot_efficient_frontier(df_overview) 

## Industry historical return rate

In [ ]:
# Convert 'date' column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Melt the returns dataframe to long format
df_long_returns = df.melt(id_vars=['Date'], var_name='stock_ticker_symbol', value_name='return_rate_5y')

# Merge the returns and industries dataframes
df_merged = pd.merge(df_long_returns, df_overview, on='stock_ticker_symbol')

# Resample to monthly and calculate mean return for each ticker
df_merged.set_index('Date', inplace=True)

df_i_share_ratio = df_merged[['industry', 'return_rate_5y', 'volatility_5y']].groupby(['industry']).resample('ME').mean()
df_i_share_ratio

Creating separate columns from indices

In [ ]:
df_i_share_ratio = df_i_share_ratio.reset_index(level=0)
df_i_share_ratio = df_i_share_ratio.reset_index(level=0)

In [ ]:
# Create line plot
fig = px.line(df_i_share_ratio, 
              x='Date', 
              y='return_rate_5y',
              color='industry', 
              title='Industry historical returns',
              labels={'Date': 'Date', 'return_rate_5y': 'Return Rate'})

# Show the plot
fig.show()

## Industry correlation

In [ ]:
# Pivot the DataFrame so that each industry is a column and the rows represent dates
df_industry_heatmap = df_i_share_ratio[['Date', 'industry', 'return_rate_5y']].pivot(index='Date', columns='industry', values='return_rate_5y')

# Ensure the columns are in a consistent order (optional)
df_industry_heatmap = df_industry_heatmap.sort_index(axis=1)

# Calculate the correlation matrix
correlation_matrix = df_industry_heatmap.corr()
correlation_matrix

In [ ]:
# Create the heatmap
fig = px.imshow(correlation_matrix,
                # labels=dict(x="Industry", y="Industry", color="Correlation"),
                x=correlation_matrix.columns,
                y=correlation_matrix.index,
                color_continuous_scale="RdBu",
                title="Industry Correlation Heatmap")

fig.layout.height = 1000
fig.layout.width = 1250

# Show the figure
fig.show()